-----------------------------------------------------------------------------------------------------------------------------------------------------------------
## Librerias

In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import re
from utils.utilities import Tiempo
import statistics
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import keyword
import xml.etree.ElementTree as ET
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que extrae los timestamps de inicio y fin de cada intentos de nivel
#### También extrae las estrellas obtenidas en cada intentos y cuantas veces se han intentado todos los niveles por cada jugador 

In [2]:
def extraerTiemposPorNivelJugador(rawData):
    
    tiempos = defaultdict(defaultdict)
    intentosNecesarios = defaultdict(defaultdict)
    inicioYFinJuego = defaultdict()
    erroresVar = defaultdict(list)
    erroresCod = defaultdict(list)
    codJugadores = defaultdict()
    codUltNivel = defaultdict()

    error1 = "No necesario crear variable"
    error2 = "Asignacion mismo valor en variable"
    error3 = "Entero o texto utilizado 2 veces"
    errorCodSucio = "Codigo muerto"

    nivelesError1Excepcion = ['variables_2', 'variables_3', 'variables_4', 'variables_6', 'variables_8', 'variables_10', 'types_2', 'basic_operators_1', 'basic_operators_3', 'basic_operators_6']
    nivelesError3Excepcion = ['tutorials_1', 'tutorials_2', 'tutorials_3', 'tutorials_4', 'tutorials_5', 'tutorials_6', 'tutorials_7', 'tutorials_8', 'tutorials_9']

    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
    
    erInitialized = re.compile(r'\binitialized$\b')
    erCompleted = re.compile(r'\bcompleted$\b')
    erAccessed = re.compile(r'\baccessed$\b')
    erProgressed = re.compile(r'\bprogressed$\b')

    erSeriousGame = re.compile(r'\bserious-game$\b')  
    erCategoryMain = re.compile(r'\bcategories_main$\b') 

    estandarNombreVar = re.compile(r'^_*[a-zA-Z][a-zA-Z0-9_]*$')

    fechaSesion = {"inicio" : None, "fin" : None}
    fechaSesion["inicio"] = rawData[0]["timestamp"]
    fechaSesion["fin"] = rawData[-1]["timestamp"]

    for evento in rawData:
        verb = evento["verb"]["id"]
        obj = evento["object"]["definition"]["type"]
        name = evento["actor"]["name"]
        timestamp = evento["timestamp"]
        objectId = evento["object"]["id"]
       
        if erLevel.search(obj): #Si el objeto de la acción es un nivel
            levelCode = erIdLevel.split(objectId)[-1]
            if levelCode != "editor_level":
                if erInitialized.search(verb): #Si la acción es inicio o reinicio
                    if "result" in evento: #Significa que ha iniciado el nivel desde el menu
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode].append({"intentos" : 1, "success" : False})
                            tiempos[name][levelCode].append({"ini" : timestamp, "fin" : None, "stars" : ""})
                        else:
                            intentosNecesarios[name][levelCode] = [{"intentos" : 1, "success" : False}]
                            tiempos[name][levelCode] = [{"ini" : timestamp, "fin" : None, "stars" : ""}]
                    else:
                        intentosNecesarios[name][levelCode][-1]["intentos"] += 1

                elif erProgressed.search(verb):
                    cod = ET.fromstring(evento['result']['extensions']['code'])
                    codJugadores[name] = cod
                    codUltNivel[name] = levelCode

                elif erCompleted.search(verb):
                    if evento["result"]["score"]["raw"] > 0 :
                        if levelCode in tiempos[name]:
                            intentosNecesarios[name][levelCode][-1]["success"] = True
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]

                        if codUltNivel[name] == levelCode:
                            try:
                                cod = codJugadores[name]
                            except:
                                print(evento)
                                print(timestamp)
                                return
                            #Analisis codigo limpio
                            antBloque = False
                            for child in cod:
                                if child.tag == "block":
                                    if antBloque:
                                        erroresCod[name].append({'error' : errorCodSucio, 'level' : levelCode})
                                        break
                                    else:
                                        antBloque = True
                                else:
                                    antBloque = False 
                            
                            #Analisis nombre variables

                            def nombreVarValido(nombresVar):
                                for nombre in nombresVar:
                                    if keyword.iskeyword(nombre):
                                        erroresCod[name].append({'error' : "Variable: '" + nombre + "' es una palabra reservada", 'level' : levelCode})
                                    elif not bool(estandarNombreVar.match(nombre)):
                                        erroresCod[name].append({'error' : "Variable: '" + nombre + "' no tiene un nombre que siga el estandar de programacion", 'level' : levelCode})

                            variablesDeclaradas = []
                            for child in cod.find("variables"):
                                variablesDeclaradas.append(child.text)
                            nombreVarValido(variablesDeclaradas)

                            #Analisis uso variables
                            vars = defaultdict()
                            values = []

                            for v in cod.find('variables'):
                                vars[v.text] = {"valores" : [], "usado" : 0}

                            for b in cod.find("block[@type='start_start']").iter('block'):
                                tarjeta = b.attrib['type']

                                if tarjeta == "text" or tarjeta == "math_number":
                                    valor = b.find('field').text
                                    if valor in values and levelCode not in nivelesError3Excepcion:
                                        erroresVar[name].append({'error' : error3, 'level' : levelCode})
                                    else:
                                        values.append(valor)

                                elif tarjeta == "variables_set":
                                    varName = b.find('field').text
                                    try:
                                        varValue = b.find('value').find('block').find('field').text
                                    except:
                                        #print(ET.tostring(cod, encoding='utf8', method='xml'))
                                        #print(ET.tostring(b, encoding='utf8', method='xml'))
                                        None                                

                                    if vars[varName]['valores'] and vars[varName]['usado'] < 2 and levelCode not in nivelesError1Excepcion:
                                        erroresVar[name].append({'error' : error1 + " " + varName, 'level' : levelCode})

                                    if varValue in vars[varName]['valores']:
                                        erroresVar[name].append({'error' : error2 + " " + varName, 'level' : levelCode})
                                    else:
                                        vars[varName]['valores'].append(varValue)

                                    vars[varName]['usado'] = 0

                                elif tarjeta == "variables_get":
                                    varName = b.find('field').text
                                    vars[varName]["usado"] += 1

                            for v in vars:
                                if vars[v]['usado'] < 2 and levelCode not in nivelesError1Excepcion:
                                    erroresVar[name].append({'error' : error1 + " " + v, 'level' : levelCode})

                    elif evento["result"]["score"]["raw"] == -1:
                        if levelCode in tiempos[name]:
                            tiempos[name][levelCode][-1]["fin"] = timestamp
                            tiempos[name][levelCode][-1]["stars"] = evento["result"]["score"]["raw"]
                    else:
                        tiempos[name][levelCode][-1]["fin"] = timestamp
        
        elif erSeriousGame.search(obj) and erInitialized.search(verb):
            if name in inicioYFinJuego:
                inicioYFinJuego[name].append({"inicio" : timestamp, "fin" : None})
            else:
                inicioYFinJuego[name] = [{"inicio" : timestamp, "fin" : None}]
        
        if not(erAccessed.search(verb) and erCategoryMain.search(objectId)):
            inicioYFinJuego[name][-1]["fin"] = timestamp
    
    return {"tiempos" : tiempos, "intentosNecesarios" : intentosNecesarios, "inicioYFinJuego" : inicioYFinJuego, "erroresVar" : erroresVar, "erroresCod" : erroresCod, "fechaSesion" : fechaSesion}

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que resta los timestamps de inicio y fin de cada intento de nivel para sacar la diferencia de tiempo

In [3]:
def tiempoPorNiveles_Jugador(data):
    tiemposJugados = defaultdict(defaultdict)
    tiemposJugadosSerializable = defaultdict(defaultdict)
    for player in data:
        for level in data[player]:
            for times in data[player][level]:
                if times["fin"] != None: #Si no se aborto el intento del nivel
                    timeDifference = Tiempo(times["ini"], times["fin"])
                    if level in tiemposJugados[player]:
                        tiemposJugados[player][level].append({"time" : timeDifference, "stars" : times["stars"]})
                        tiemposJugadosSerializable[player][level].append({"time" : timeDifference.toString(), "stars" : times["stars"]})
                    else:
                        tiemposJugados[player][level] = [{"time" : timeDifference, "stars" : times["stars"]}]
                        tiemposJugadosSerializable[player][level] = [{"time" : timeDifference.toString(), "stars" : times["stars"]}]
    return tiemposJugados, tiemposJugadosSerializable

In [4]:
def tiempoTotalJuego(inicioYFinJuego):
    tiempoTotal = defaultdict()
    for player in inicioYFinJuego:
        for time in inicioYFinJuego[player]:
            if time["fin"] != None:
                if player in tiempoTotal:
                    tiempoTotal[player] += Tiempo(time["inicio"], time["fin"])
                else:
                    tiempoTotal[player] = Tiempo(time["inicio"], time["fin"])

    for p in tiempoTotal:
        tiempoTotal[p] = str(tiempoTotal[p])
    return tiempoTotal

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que devuelve el tiempo medio necesario para completar un nivel y un diccionario con el tiempo empleado por cada jugador hasta completar el nivel
(El tiempo necesario para cada jugador es la suma de todos los tiempos hasta conseguir completar el nivel)
* También devuelve la media de estrellas y una lista de los niveles ordenados

In [25]:
#TAMBIEN DEVUELVE EL TIEMPO INDIVIDUAL DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getMediaTiempoPorNivel(tiempos, soloPrimerExito = True, tiemposOrdenados = False):
    medias = defaultdict(list)
    mediasEstrellas = defaultdict(list)
    tiempoCompletarNivelIndividual = defaultdict(defaultdict)
    
    for player in tiempos:
        for level in tiempos[player]:
            tAux = Tiempo("0s")
            for t in tiempos[player][level]:
                if t["stars"] != "" and int(t["stars"]) != -1:
                    if level in medias:
                        medias[level].append(int(tAux + t["time"]))
                        mediasEstrellas[level].append(int(t["stars"]))
                    else:
                        medias[level] = [int(tAux + t["time"])]
                        mediasEstrellas[level] = [int(t["stars"])]
                        
                    tiempoCompletarNivelIndividual[player][level] = tAux + t["time"]
                    if soloPrimerExito:
                        break
                else:
                    tAux += t["time"]
    for m in medias:
        medias[m] = Tiempo(str(int(round(statistics.mean(medias[m]), 0))) + "s")
        
    for m in mediasEstrellas:
        mediasEstrellas[m] = statistics.mean(mediasEstrellas[m])
    
    listaNiveles = medias.keys()
    
    if tiemposOrdenados:
        medias = sorted(medias.items(), key=lambda x: x[1])
        mediasEstrellas = sorted(mediasEstrellas.items(), key=lambda x: x[1])
    else:
        medias = list(medias.items())
        mediasEstrellas = list(mediasEstrellas.items())
    
    return {"mediaTiempos" : medias, "mediaEstrellas" : mediasEstrellas, "listaNiveles" : listaNiveles, "tiemposIndividuales" : tiempoCompletarNivelIndividual}

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Devuelve un diccionario con clave: id Jugador y valor: ultimo nivel alcanzado

In [6]:
def getUltimoNivelAlcanzado(tiempos):
    ultNivel = defaultdict()
    for player in tiempos:
        for level in tiempos[player]:
            ultNivel[player] = level
    return ultNivel

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Devuelve los intentos necesarios de cada jugador para completar un nivel al igual que los intentos medios

In [7]:
#TAMBIEN DEVUELVE LOS INTENTOS INDIVIDUALES DE CADA JUGADOR PARA COMPLETAR CADA NIVEL
def getIntentosMedios_HastaCompletarNivel(intentosNecesarios, intentosOrdenados = False):
    intentosMedios = defaultdict(list)
    intentosCompletarNivelIndividual = defaultdict(defaultdict)
    
    for name in intentosNecesarios:
        for level in intentosNecesarios[name]:
            cont = 0
            for i in intentosNecesarios[name][level]:
                cont += i["intentos"]
                if i["success"] == True:
                    intentosMedios[level].append(cont)
                    intentosCompletarNivelIndividual[name][level] = cont
                    break
            
    for level in intentosMedios:
        intentosMedios[level] = round(statistics.mean(intentosMedios[level]), 2)
    
    if intentosOrdenados:
        intentosMedios = sorted(intentosMedios.items(), key=lambda x: x[1])
    else:
        intentosMedios = list(intentosMedios.items())
    return {"intentosMedios" : intentosMedios, "intentosIndividual" : intentosCompletarNivelIndividual}

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Transforma un diccionario de diccionarios en un diccionario de arrays
* Se utiliza para tener una lista de todos los tiempos de los usuarios cuando tenemos un diccionario dentro de otro con primera clave nombre usuario y segunda clave codigo del nivel

In [8]:
def extraerArray(my_dict):
    nested_keys = []
    nested_values = []

    for k, v in my_dict.items():
        for nested_k, nested_v in v.items():
            nested_keys.append(nested_k)
            nested_values.append(nested_v)

    # create a dictionary with keys as the values of the nested_keys array
    nested_dict = defaultdict(list)
    for key, value in zip(nested_keys, nested_values):
        nested_dict[key].append(value)
    
    return nested_dict

In [9]:
def extraerArrayConNombres(my_dict):
    nested_keys = []
    nested_values = []

    for k, v in my_dict.items():
        for nested_k, nested_v in v.items():
            nested_keys.append(nested_k)
            nested_values.append({"name" : k, "valor" : nested_v})

    # create a dictionary with keys as the values of the nested_keys array
    nested_dict = defaultdict(defaultdict)
    for key, value in zip(nested_keys, nested_values):
        nested_dict[key][value["name"]] = value["valor"]
    
    return nested_dict

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Recibe un diccionario con clave nivel y valor una lista de tiempos y parsea los tiempos a integer

In [10]:
def parseTiemposDictToInteger(data_dict):
    parsed_dict = {}

    for key, value in data_dict.items():
        parsed_list = []
        for element in value:
            parsed_list.append(int(element))
        parsed_dict[key] = parsed_list

    return parsed_dict

-----------------------------------------------------------------------------------------------------------------------------------------------------------------
### Función que genera los boxplots recibiendo un diccionario con clave nivel y valor una lista de enteros

In [11]:
def create_boxplots(data_dict):
    # Group levels by category
    category_dict = {}
    for level, data in data_dict.items():
        category = level.split('_')[0]
        if category in category_dict:
            category_dict[category].append(data)
        else:
            category_dict[category] = [data]
    
    # Create boxplots for each category and add relevant information
    boxplots = []
    labels = []
    for i, (category, data_list) in enumerate(category_dict.items()):
        fig, ax = plt.subplots()
        bp = ax.boxplot(data_list, showfliers=True, patch_artist=True, flierprops={'marker': '+', 'markerfacecolor': '#738FA7'})
        ax.set_title(category)

        color = '#738FA7'
        for patch in bp['boxes']:
            patch.set_facecolor(color)
        
        boxplots.append(bp)
        labels.append(category)
    
    # Set tick labels
    fig, ax = plt.subplots()
    ax.set_xticks(range(len(labels)))
    ax.set_xticklabels(labels)
    ax.tick_params(axis='x', labelrotation=45)
    
    return boxplots


In [12]:
def getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles):
    cuantosHanLlegadoAlNivel = defaultdict()
    
    ultNivelCopia = ultNivelCompletado.copy()

    #Contamos cuanta gente ha llegado hasta cada nivel
    for nivel in niveles:
        cuantosHanLlegadoAlNivel[nivel] = len(ultNivelCopia)
        
        keys_to_delete = [k for k, v in ultNivelCopia.items() if v == nivel]
        for k in keys_to_delete:
            del ultNivelCopia[k]

    return cuantosHanLlegadoAlNivel

In [13]:
def generateChartNivelesAlcanzados(niveles, ultNivelCompletado):
    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)
    category_dict = {}
    for level, value in cuantosHanLlegadoAlNivel.items():
        category = level.split('_')[0]
        level_number = int(level.split('_')[-1])
        if category in category_dict:
            category_dict[category].append((level_number, value))
        else:
            category_dict[category] = [(level_number, value)]
    for category, levels in category_dict.items():
        category_dict[category] = sorted(levels, key=lambda x: x[0])

    # Select the last level for each category
    last_levels = {}
    for category, levels in category_dict.items():
        last_level = levels[-1][0]
        last_levels[category] = levels[-1][1]

    # Create bar chart and add labels
    fig, ax = plt.subplots()
    ax.bar(last_levels.keys(), last_levels.values(), color="#738FA7")
    ax.set_xticklabels(last_levels.keys(), rotation=45, ha='right')
        
    plt.show()

In [14]:
def generateChartNivelesAlcanzados2(niveles, ultNivelCompletado):
    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)
    print(cuantosHanLlegadoAlNivel)
    ultNivelCat = {}
    for level in cuantosHanLlegadoAlNivel:
        ultNivelCat[(" ".join(level.split("_")[:-1])).capitalize()] = cuantosHanLlegadoAlNivel[level]
    
    df = pd.DataFrame({"levels" : list(ultNivelCat.keys()), "nJugadores" : list(ultNivelCat.values())})
    fig = px.bar(df, x="levels", y='nJugadores', labels={'levels':'Categorías', 'nJugadores':'Numero Jugadores'})
    fig.update_layout(plot_bgcolor='#C3CEDA')
    fig.update_traces(marker_color='#738FA7', hovertemplate='<b>Numero Jugadores: %{y}</b>')
    #fig.write_json("./plots/categorias.json")
    return fig


In [15]:
def getChartPorcentajeCategorias(niveles, ultNivelCompletado, total_players):

    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)

    ultimoNivelCat = {}
    for level in niveles:
        category = level.split("_")[0]
        ultimoNivelCat[category] = level
    
    percentages = {}
    for ultNivel in ultimoNivelCat.values():
        percentages[ultNivel] = (cuantosHanLlegadoAlNivel[ultNivel]/total_players)*100
    
    fig, ax = plt.subplots()
    ax.bar(range(len(percentages)), list(percentages.values()), align='center', color="#738FA7")
    ax.set_xticks(range(len(percentages)))
    ax.set_xticklabels(map(lambda c: c.split("_")[0], list(percentages.keys())), rotation=45, ha='right')
    ax.set_ylim((0, 100))
    ax.set_ylabel('Porcentaje Jugadores')
    ax.set_xlabel('Categorías')
    ax.set_title('Categorías Superadas')
    
    return ax

In [16]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [17]:
#JSONFile = open('../Web/datos/1/trazasOrdenadas.json')
JSONFile = open('../Web/datos/2/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

resultados_Tiempos_Nivel_Jugador = extraerTiemposPorNivelJugador(rawData)

#print(resultados_Tiempos_Nivel_Jugador["erroresCod"])
with open("erroresCod.json", "w") as f:
    json.dump(resultados_Tiempos_Nivel_Jugador["erroresCod"], f)

In [18]:
tiempoTJuego = tiempoTotalJuego(resultados_Tiempos_Nivel_Jugador["inicioYFinJuego"])
json.dumps(tiempoTJuego)
#pd.DataFrame(tiempoTJuego, index = ["Tiempo"])

'{"cwsgn": "1h/48m/56s", "qwwpu": "1h/48m/48s", "ntjpz": "1h/48m/30s", "xirlo": "1h/48m/7s", "auqln": "1h/48m/1s", "oldmh": "1h/44m/20s", "hrwjz": "1h/46m/42s", "cwacw": "1h/27m/35s", "kumtj": "1h/27m/4s", "azrda": "1h/47m/1s", "rtjhk": "1h/26m/53s", "wrjij": "1h/46m/24s", "srobt": "1h/44m/12s", "cvcyk": "1h/46m/10s", "pmnsg": "1h/46m/22s", "zxtvi": "1h/29m/44s", "wxqdk": "1h/46m/12s", "ueoff": "1h/45m/59s", "rqluq": "1h/45m/54s", "ixqal": "1h/26m/18s", "fomzb": "1h/46m/12s", "kmhsu": "1h/45m/41s", "mopue": "1h/26m/4s", "dymxy": "1h/27m/50s", "zbmtt": "1h/28m/58s", "jbnwn": "1h/31m/26s", "ijyql": "1h/48m/1s", "cuvsb": "1h/27m", "zmkfk": "1h/31m/23s", "dcsor": "1h/30m/5s", "lqevy": "1h/25m/49s", "bfqzk": "1h/27m/35s", "cqylj": "1h/27m/14s", "foqfy": "1h/29m/20s", "nhvyj": "1h/22m/12s", "dbfzw": "1h/19m/57s", "acwwx": "57m/27s", "nwyig": "1h/11m/9s"}'

In [19]:
tiemposIntentosJugadores, p = tiempoPorNiveles_Jugador(resultados_Tiempos_Nivel_Jugador["tiempos"])
with open("prueba.json", "w") as f:
    json.dump({"tiempo" : p, "intentos" : resultados_Tiempos_Nivel_Jugador["intentosNecesarios"]}, f)
pd.DataFrame(tiemposIntentosJugadores)
#pd.DataFrame(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"])

,qwwpu,cwsgn,ntjpz,xirlo,auqln,oldmh,cwacw,hrwjz,azrda,kumtj,wrjij,cvcyk,zxtvi,pmnsg,fomzb,rqluq,rtjhk,ixqal,wxqdk,srobt,kmhsu,mopue,ueoff,dymxy,jbnwn,zbmtt,ijyql,zmkfk,dcsor,bfqzk,cqylj,lqevy,foqfy,cuvsb,nhvyj,dbfzw,acwwx,nwyig
tutorials_1,"[{'time': 25s, 'stars': 3}]","[{'time': 26s, 'stars': 3}]","[{'time': 20s, 'stars': 3}]","[{'time': 31s, 'stars': 3}]","[{'time': 3m/18s, 'stars': 2}]","[{'time': 16s, 'stars': 3}]","[{'time': 1m/33s, 'stars': 2}, {'time': 4s, 'stars': 3}]","[{'time': 23s, 'stars': 3}]","[{'time': 33s, 'stars': 3}]","[{'time': 23s, 'stars': 3}]","[{'time': 19s, 'stars': 3}]","[{'time': 44s, 'stars': 3}]","[{'time': 1m/57s, 'stars': 2}]","[{'time': 59s, 'stars': 2}]","[{'time': 22s, 'stars': 3}]","[{'time': 59s, 'stars': 2}, {'time': 5s, 'stars': 3}]","[{'time': 54s, 'stars': 2}]","[{'time': 17s, 'stars': 3}]","[{'time': 50s, 'stars': 3}]","[{'time': 4m/15s, 'stars': 2}]","[{'time': 35s, 'stars': 3}]","[{'time': 3m/9s, 'stars': 3}]","[{'time': 3m/9s, 'stars': -1}, {'time': 1m/10s, 'stars': 2}]","[{'time': 56s, 'stars': 3}]","[{'time': 43s, 'stars': 3}]","[{'time': 2m/3s, 'stars': 2}]","[{'time': 51s, 'stars': -1}, {'time': 5s, 'stars': 3}]","[{'time': 27s, 'stars': 3}]","[{'time': 16s, 'stars': 3}]","[{'time': 1m/25s, 'stars': 2}]","[{'time': 3m/59s, 'stars': 3}]","[{'time': 1m/20s, 'stars': 2}, {'time': 4s, 'stars': 3}]","[{'time': 47s, 'stars': 3}]","[{'time': 1m/29s, 'stars': 2}, {'time': 7s, 'stars': 3}]","[{'time': 23s, 'stars': 3}]","[{'time': 58s, 'stars': 2}]","[{'time': 35s, 'stars': 2}]","[{'time': 15s, 'stars': 3}]"
tutorials_2,"[{'time': 14s, 'stars': 3}]","[{'time': 15s, 'stars': 3}]","[{'time': 25s, 'stars': 2}, {'time': 12s, 'stars': 3}]","[{'time': 19s, 'stars': 3}]","[{'time': 25s, 'stars': 2}]","[{'time': 17s, 'stars': 3}]","[{'time': 19s, 'stars': 3}]","[{'time': 1m/18s, 'stars': 3}]","[{'time': 34s, 'stars': 2}, {'time': 14s, 'stars': 3}]","[{'time': 16s, 'stars': 3}]","[{'time': 20s, 'stars': 3}]","[{'time': 32s, 'stars': 2}]","[{'time': 20s, 'stars': 3}]","[{'time': 18s, 'stars': 3}]","[{'time': 1m/10s, 'stars': 2}, {'time': 10s, 'stars': 3}]","[{'time': 24s, 'stars': 2}, {'time': 10s, 'stars': 3}]","[{'time': 23s, 'stars': 3}]","[{'time': 21s, 'stars': 2}, {'time': 12s, 'stars': 3}]","[{'time': 28s, 'stars': 3}]","[{'time': 24s, 'stars': 3}]","[{'time': 21s, 'stars': 3}]","[{'time': 1m/7s, 'stars': 2}]","[{'time': 24s, 'stars': 3}]","[{'time': 50s, 'stars': 2}]","[{'time': 19s, 'stars': 3}]","[{'time': 21s, 'stars': 3}]","[{'time': 1m/20s, 'stars': 2}]","[{'time': 23s, 'stars': 3}]","[{'time': 25s, 'stars': 2}, {'time': 9s, 'stars': 3}]","[{'time': 22s, 'stars': 3}]","[{'time': 18s, 'stars': 3}]","[{'time': 51s, 'stars': 2}, {'time': 13s, 'stars': 3}]","[{'time': 27s, 'stars': 3}]","[{'time': 34s, 'stars': 2}, {'time': 11s, 'stars': 3}]","[{'time': 18s, 'stars': 3}]","[{'time': 15s, 'stars': 3}]","[{'time': 25s, 'stars': 2}]","[{'time': 20s, 'stars': 3}]"
tutorials_3,"[{'time': 15s, 'stars': 3}]","[{'time': 17s, 'stars': 3}]","[{'time': 15s, 'stars': 3}, {'time': 5s, 'stars': -1}]","[{'time': 15s, 'stars': 3}]","[{'time': 19s, 'stars': 2}]","[{'time': 19s, 'stars': 3}]","[{'time': 23s, 'stars': 3}]","[{'time': 15s, 'stars': 3}]","[{'time': 25s, 'stars': 3}]","[{'time': 18s, 'stars': 3}]","[{'time': 18s, 'stars': 3}]","[{'time': 29s, 'stars': 3}]","[{'time': 24s, 'stars': 3}]","[{'time': 21s, 'stars': 3}]","[{'time': 18s, 'stars': 3}]","[{'time': 13s, 'stars': 3}]","[{'time': 19s, 'stars': 3}]","[{'time': 29s, 'stars': 3}]","[{'time': 17s, 'stars': 3}]","[{'time': 18s, 'stars': 2}]","[{'time': 34s, 'stars': 3}]","[{'time': 1m/21s, 'stars': 3}]","[{'time': 34s, 'stars': 3}]","[{'time': 18s, 'stars': 3}]","[{'time': 19s, 'stars': 3}]","[{'time': 20s, 'stars': 3}]","[{'time': 1m/38s, 'stars': 2}]","[{'time': 16s, 'stars': 3}]","[{'time': 16s, 'stars': 3}]","[{'time': 16s, 'stars': 3}]","[{'time': 17s, 'stars': 3}]","[{'time': 24s, 'stars': 3}]","[{'time': 23s, 'stars': 3}]","[{'time': 12s

In [26]:
soloPrimerExito = True
tiemposOrdenados = False
tiemposMedios = getMediaTiempoPorNivel(tiemposIntentosJugadores, soloPrimerExito, tiemposOrdenados)
#pd.DataFrame(tiemposMedios["mediaTiempos"])
#pd.DataFrame(tiemposMedios["mediaEstrellas"])
#pd.DataFrame(tiemposMedios["listaNiveles"])
#pd.DataFrame(tiemposMedios["tiemposIndividuales"])

In [206]:
ultNivelAlcanzado = getUltimoNivelAlcanzado(tiemposMedios["tiemposIndividuales"])
#print(ultNivelAlcanzado)

In [207]:
intentosOrdenados = False
intentosMedios_Individual = getIntentosMedios_HastaCompletarNivel(resultados_Tiempos_Nivel_Jugador["intentosNecesarios"], intentosOrdenados)
#pd.DataFrame(intentosMedios_Individual["intentosMedios"])
#pd.DataFrame(intentosMedios_Individual["intentosIndividual"])

In [208]:
tiemposList = extraerArray(tiemposMedios["tiemposIndividuales"])
intentosList = extraerArray(intentosMedios_Individual["intentosIndividual"])
#pd.DataFrame.from_dict(tiemposList, orient='index').transpose()
#pd.DataFrame.from_dict(intentosList, orient='index').transpose()

In [209]:
intentosListNombres = extraerArrayConNombres(intentosMedios_Individual["intentosIndividual"])
tiemposListNombres = extraerArrayConNombres(tiemposMedios["tiemposIndividuales"])
#pd.DataFrame.from_dict(intentosListNombres, orient='index').transpose()
#pd.DataFrame.from_dict(tiemposListNombres, orient='index').transpose()

In [210]:
def parseTiemposDictConNombresToInteger(tiemposDict):
    for l in tiemposDict:
        for j in tiemposDict[l]:
            tiemposDict[l][j] = int(tiemposDict[l][j])
    return tiemposDict

In [211]:
fig = generateChartNivelesAlcanzados2(tiemposMedios["listaNiveles"], ultNivelAlcanzado)
fig.show()

defaultdict(None, {'tutorials_1': 145, 'tutorials_2': 144, 'tutorials_3': 144, 'tutorials_4': 143, 'tutorials_5': 143, 'tutorials_6': 143, 'tutorials_7': 143, 'variables_1': 143, 'variables_2': 142, 'variables_3': 140, 'variables_4': 137, 'variables_5': 133, 'variables_6': 130, 'variables_7': 123, 'variables_8': 117, 'variables_9': 114, 'types_1': 113, 'types_2': 111, 'types_3': 103, 'types_4': 89, 'types_5': 77, 'types_6': 70, 'types_7': 56, 'types_8': 46, 'types_9': 38, 'basic_operators_1': 31, 'basic_operators_2': 29, 'basic_operators_3': 16, 'basic_operators_4': 12, 'basic_operators_5': 8, 'basic_operators_6': 7, 'basic_operators_7': 6, 'loops_1': 6, 'loops_2': 6, 'loops_3': 6})


In [212]:
#getChartPorcentajeCategorias(tiemposMedios["listaNiveles"], ultNivelAlcanzado, len(ultNivelAlcanzado))

In [213]:
#boxplotsIntentos = create_boxplots(intentosList)

In [214]:
#boxplotsTiempos = create_boxplots(parseTiemposDictToInteger(tiemposList))

In [215]:
def create_boxplots3(data_dict, titulo):
    categorias = defaultdict(defaultdict)
    for d in data_dict:
        categorias[d.split("_")[0]][d] = data_dict[d]

    boxplots = []
    for c in categorias:
        df = pd.DataFrame.from_dict(categorias[c], orient="index")
        df = df.reset_index()
        df = df.rename(columns={'index' : 'Niveles'})
        df_melted = df.melt(id_vars=['Niveles'], var_name='Jugador', value_name=titulo)
        # create boxplot
        fig = px.box(df_melted, x='Niveles', y=titulo, hover_name='Jugador')
        fig.update_layout(plot_bgcolor='#C3CEDA')
        fig.update_traces(marker_color='#738FA7', hovertemplate='<b>%{hovertext}</b><br>' + titulo + ': %{y}')
        #fig.write_json("./plots/"+c + "_"+ titulo +".json")
        boxplots.append(fig)
    return boxplots

In [216]:
boxsIntentos= create_boxplots3(intentosListNombres, 'Intentos')
for b in boxsIntentos:
    #b.show()
    None

In [217]:
boxsTiempos = create_boxplots3(parseTiemposDictConNombresToInteger(tiemposListNombres), "Tiempo(s)")
for b in boxsTiempos:
    #b.show()
    None

In [218]:
def getChartComparativas(niveles, tiemposMedios, ultNivelCompletado, jugClase):
    cuantosHanLlegadoAlNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelCompletado, niveles)
    ultNivelCat = {}
    for level in cuantosHanLlegadoAlNivel:
        ultNivelCat[(" ".join(level.split("_")[:-1])).capitalize()] = cuantosHanLlegadoAlNivel[level]

    for c in ultNivelCat:
        ultNivelCat[c] = round((ultNivelCat[c]/jugClase)*100, 2)

    categorias = defaultdict(defaultdict)
    for d in tiemposMedios:
        categorias[(" ".join(d[0].split("_")[:-1])).capitalize()][d[0]] = int(d[1])

    ### Actualizar datos globales
    JSONFile = open('datosGlobales.json')
    datosGlobales = json.load(JSONFile)
    JSONFile.close()

    if( 1 not in datosGlobales["institutos"]):
        jugTotales = jugClase + datosGlobales["numeroAlumnos"]
        for c in datosGlobales["porcentaje"]:
            if c in ultNivelCat:
                datosGlobales["porcentaje"][c] = (datosGlobales["porcentaje"][c]*datosGlobales["numeroAlumnos"]/jugTotales) + (ultNivelCat[c]*jugClase/jugTotales)

        
        datosGlobales["numeroAlumnos"] = jugTotales        
        datosGlobales["institutos"].append(1)

        for c in datosGlobales["tiempoMedio"]:
            if c in categorias:
                for l in datosGlobales["tiempoMedio"][c]:
                    if l in categorias[c]:
                        datosGlobales["tiempoMedio"][c][l] = (datosGlobales["tiempoMedio"][c][l]*datosGlobales["numeroAlumnos"]/jugTotales) + (categorias[c][l]*jugClase/jugTotales)
                        
        with open('datosGlobales.json', 'w') as json_file:
            json.dump(datosGlobales, json_file)


    print(ultNivelCat)
    fig = go.Figure(data=[
        go.Bar(name="Clase", x=list(ultNivelCat.keys()), y=list(ultNivelCat.values()), marker_color="#738FA7",
               hovertemplate='<b>%{y}%</b>'),
        go.Bar(name="Global", x=list(datosGlobales["porcentaje"].keys()), y=list(datosGlobales["porcentaje"].values()), marker_color="#0C4160",
               hovertemplate='<b>%{y}%</b>'),
    ])
    fig.update_layout(plot_bgcolor='#C3CEDA')
    fig.update_layout(barmode='group')
    fig.update_layout(title_text='Porcentaje Categorías Superadas VS Global')
    #fig.write_json("./plots/porcentajeCategorias.json")
    fig.show()


    print(categorias)
    for c in datosGlobales["tiempoMedio"]:
        if c in categorias:
            fig = go.Figure(data=[
                go.Bar(name = "Clase", x=list(categorias[c].keys()), y=list(categorias[c].values()), marker_color="#738FA7",
                    hovertemplate='<b>%{y}s</b>'),
                go.Bar(name = "Global", x=list(datosGlobales["tiempoMedio"][c].keys()), y=list(datosGlobales["tiempoMedio"][c].values()), marker_color="#0C4160",
                    hovertemplate='<b>%{y}s</b>')
            ])
            fig.update_layout(plot_bgcolor='#C3CEDA')
            fig.update_layout(barmode='group')
            fig.write_json("./plots/" + c.split(" ")[0].lower() + "_mediaTiemposComparativa.json")
            fig.show()

In [219]:
#getChartComparativas(tiemposMedios["listaNiveles"], tiemposMedios["mediaTiempos"], ultNivelAlcanzado, len(ultNivelAlcanzado))

In [220]:

def getListaCategorias(listaNiveles):
    categorias = defaultdict()
    erroresVar = defaultdict()
    erroresCod = defaultdict()

    for n in listaNiveles:
        categorias[n.split("_")[0]] = (" ".join(n.split("_")[:-1])).capitalize()
    
    for j in resultados_Tiempos_Nivel_Jugador["erroresVar"]:
        for e in resultados_Tiempos_Nivel_Jugador["erroresVar"][j]:
            if e["level"] in erroresVar:
                erroresVar[e["level"]] += 1
            else:
                erroresVar[e["level"]] = 1

    for j in resultados_Tiempos_Nivel_Jugador["erroresCod"]:
        for e in resultados_Tiempos_Nivel_Jugador["erroresCod"][j]:
            if e["level"] in erroresCod:
                erroresCod[e["level"]] += 1
            else:
                erroresCod[e["level"]] = 1
    
    nPersonasNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelAlcanzado, tiemposMedios["listaNiveles"])
    listVar = []
    listCod = []
    for n in nPersonasNivel:
        if n in erroresVar:
            listVar.append({n : {"errores" : erroresVar[n], "jugadores" : nPersonasNivel[n]}})
        else:
            listVar.append({n : {"errores" : 0, "jugadores" : nPersonasNivel[n]}})

        if n in erroresCod:
            listCod.append({n : {"errores" : erroresCod[n], "jugadores" : nPersonasNivel[n]}})
        else:
            listCod.append({n : {"errores" : 0, "jugadores" : nPersonasNivel[n]}})


    listVar = sorted(listVar, key=lambda x: x[list(x.keys())[0]]['errores'], reverse=True)
    listCod = sorted(listCod, key=lambda x: x[list(x.keys())[0]]['errores'], reverse=True)

    with open('./info.json', 'w') as json_file:
        json.dump({"categorias" : categorias, "niveles" : list(listaNiveles), "nErroresVar" : listVar, "nErroresCod" : listCod, "fechaSesion" : resultados_Tiempos_Nivel_Jugador["fechaSesion"], "nJugadores" : len(ultNivelAlcanzado)}, json_file)
    return categorias

getListaCategorias(tiemposMedios["listaNiveles"])

defaultdict(None,
            {'tutorials': 'Tutorials',
             'variables': 'Variables',
             'types': 'Types',
             'basic': 'Basic operators',
             'loops': 'Loops'})

In [221]:
JSONFile = open('../Web/datos/1/trazasOrdenadas.json')
rawData = json.load(JSONFile)
JSONFile.close()

erIdLevel = re.compile(r'/')
erLevel = re.compile(r'\blevel$\b')

for e in rawData:
    obj = e["object"]["definition"]["type"]
    objectId = e["object"]["id"]
    if erLevel.search(obj):
        levelCode = erIdLevel.split(objectId)[-1]
        if e["actor"]["name"] == "bdwyj" and levelCode == "variables_9":
            #print(e)
            #print("-----------------------------------------------------------------")
            None

In [222]:
nPersonasNivel = getCuantasPersonasHanAlcanzadoNivel(ultNivelAlcanzado, tiemposMedios["listaNiveles"])
print(nPersonasNivel)

defaultdict(None, {'tutorials_1': 145, 'tutorials_2': 144, 'tutorials_3': 144, 'tutorials_4': 143, 'tutorials_5': 143, 'tutorials_6': 143, 'tutorials_7': 143, 'variables_1': 143, 'variables_2': 142, 'variables_3': 140, 'variables_4': 137, 'variables_5': 133, 'variables_6': 130, 'variables_7': 123, 'variables_8': 117, 'variables_9': 114, 'types_1': 113, 'types_2': 111, 'types_3': 103, 'types_4': 89, 'types_5': 77, 'types_6': 70, 'types_7': 56, 'types_8': 46, 'types_9': 38, 'basic_operators_1': 31, 'basic_operators_2': 29, 'basic_operators_3': 16, 'basic_operators_4': 12, 'basic_operators_5': 8, 'basic_operators_6': 7, 'basic_operators_7': 6, 'loops_1': 6, 'loops_2': 6, 'loops_3': 6})


In [231]:
def valorUltNivelAlcanzado(ultNivelAlcanzado, listaNiveles):
    valorUltNivel = defaultdict()
    listaNiveles = list(listaNiveles)
    for j in ultNivelAlcanzado:
        valorUltNivel[j] = listaNiveles.index(ultNivelAlcanzado[j])
    return valorUltNivel
print(valorUltNivelAlcanzado(ultNivelAlcanzado, tiemposMedios["listaNiveles"]))

defaultdict(None, {'gqoj': 10, 'hgfx': 0, 'uala': 2, 'pbgky': 23, 'auvqa': 22, 'qeqyj': 24, 'fkmtp': 24, 'bdwyj': 19, 'eprdu': 34, 'hjpbw': 18, 'ftpnb': 34, 'gleqe': 24, 'ikcar': 21, 'loemd': 26, 'gcpze': 34, 'khadt': 26, 'zxvkj': 21, 'srjon': 26, 'uzzpo': 26, 'vsxld': 25, 'sxipu': 18, 'kmqou': 21, 'ejeuw': 20, 'xnpqf': 21, 'ipdcp': 21, 'epqkm': 12, 'zwzfv': 22, 'krahz': 26, 'grkij': 22, 'zcibf': 22, 'rdiaf': 27, 'yvpmf': 23, 'pygoc': 29, 'ubmtx': 20, 'edxmq': 19, 'asopk': 34, 'onyep': 26, 'swvxx': 18, 'kkoif': 27, 'oahey': 28, 'cmbfi': 30, 'bhyhw': 27, 'accul': 19, 'tekwp': 20, 'dunul': 28, 'dwqwi': 26, 'yfohf': 21, 'nmveq': 12, 'tjfzy': 19, 'sjpkc': 18, 'camhi': 12, 'stiry': 18, 'ewqbs': 21, 'lwryl': 18, 'fivin': 19, 'psido': 21, 'tinco': 13, 'mymho': 11, 'zbgwk': 14, 'fyyya': 13, 'gdljy': 16, 'wjaxi': 26, 'ucdvy': 23, 'dvmlu': 12, 'yxewz': 18, 'mjiso': 24, 'gguuc': 19, 'kkliq': 23, 'dmwvy': 9, 'hpdxj': 20, 'tgwmn': 13, 'hqmhq': 17, 'rziia': 17, 'yxjji': 11, 'ztyed': 12, 'vddlg': 9, 

In [223]:
#Si tenerEnCuentaMove = False -> cuando se realiza una accion de tipo "move" no se tiene en cuenta para calcular el tiempo entre interacciones, como si no existiese la traza
def extraerTiemposEntreInteracciones2(rawData, tenerEnCuentaMove = False):
    ultimaInteraccionJugador = defaultdict()
    
    tiempoInicio_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre inicio de nivel y la siguiente interaccion (reinicio, completado o interaccion con tarjeta)
    tiempoInteraccion_Completado_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion (inicio, reinicio o interaccion con tarjeta) y terminar el nivel
    tiempoInteraccion_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion con tarjeta y otra interaccion con tarjeta
    
    anomalias = []
    
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erLevelExitButton = re.compile(r'\blevel_exit_button$\b') 
    erInitialized = re.compile(r'\binitialized$\b')
    erSeriousGame = re.compile(r'\bserious-game$\b')    
    erCompleted = re.compile(r'\bcompleted$\b')
    erLevel = re.compile(r'\blevel$\b')

        
    for e in rawData:
        verb = e["verb"]["id"]
        obj = e["object"]["definition"]["type"]
        objectId = e["object"]["id"]
        name = e["actor"]["name"]
        timestamp = e["timestamp"]
        
        erIdLevel = re.compile(r'/')
        levelCode = erIdLevel.split(objectId)[-1]
        
        if(levelCode != "editor_level"):
            
            if erSeriousGame.search(obj) and erInitialized.search(verb): #Si inicia el juego se reinicia su ultima interaccion
                if name in ultimaInteraccionJugador:
                    del ultimaInteraccionJugador[name]

            elif erLevel.search(obj):
                if erInitialized.search(verb): #Significa que ha iniciado o reiniciado el nivel
                    if "result" in e: #Inicia el nivel desde el menu
                        ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "inicio"}
                    else: #Reinicia el nivel
                        try:
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            
                            if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                                if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "reinicio"})
                                else:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "reinicio"}]
                            #Actualizamos ultima interaccion
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "reinicio"}
                        except:
                            #Si entra aqui es porque ha reiniciado un nivel que no habia iniciado
                            #Trazas estan mal
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha reiniciado un nivel que no estaba iniciado"})

                elif erCompleted.search(verb) and e["result"]["score"]["raw"] != 0:
                    #Si completa un nivel o sale al menu se reinicia su ultima interaccion, si falla el intento pero vuelve a intentarlo no se reinicia
                    #Si raw == 0 significa que se reinicia el nivel
                    try:
                        t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)                        
                        #Añadimos la dif de tiempo al diccionario del jugador
                        if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                            if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]})
                            else:
                                tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "terminado", "exito" : e["result"]["success"]}]
                                
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "inicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "inicio"}]
                                    
                        elif ultimaInteraccionJugador[name]["accion"] == "reinicio":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "reinicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "reinicio"}]

                        elif ultimaInteraccionJugador[name]["accion"] == "interaccion":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]
                        #Actualizamos ultima interaccion, la borramos
                        del ultimaInteraccionJugador[name]
                    except:
                        #Si entra aqui es porque se ha completado un nivel que no se habia iniciado
                        #Probablemente por anomalias en las trazas
                        anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo": "Se ha completado un nivel que no estaba iniciado"})

            elif erGameObject.search(obj) and erInteracted.search(verb):
                if "result" in e and "extensions" in e["result"] and "level" in e["result"]["extensions"]:
                    levelCode = e["result"]["extensions"]["level"]
                    if not erLevelExitButton.search(objectId) and levelCode != "editor_level":
                        if name in ultimaInteraccionJugador:
                            if e["result"]["extensions"]["action"] != "move" or tenerEnCuentaMove: #<-- Que accion realiza
                                t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                                #Añadimos la dif de tiempo al diccionario del jugador

                                if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                    if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion" : "interaccionTarjeta"})
                                    else:
                                        tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion" : "interaccionTarjeta"}]

                                if ultimaInteraccionJugador[name]["accion"] == "interaccionTarjeta":
                                    if levelCode in tiempoInteraccion_Interaccion_Jugador[name]:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode].append({"tiempo" : t, "accion_anterior" : "interaccionTarjeta"})
                                    else:
                                        tiempoInteraccion_Interaccion_Jugador[name][levelCode] = [{"tiempo" : t, "accion_anterior" : "interaccionTarjeta"}]

                                ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "interaccionTarjeta"}
                        else:
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha interactuado en un nivel que no estaba iniciado"})
                        
                    
    # Dataframe 1: tiempoInteraccion_Completado_Jugador
    df_tiempoInteraccion_Completado_Jugador = pd.DataFrame(tiempoInteraccion_Completado_Jugador)
    df_tiempoInteraccion_Completado_Jugador = df_tiempoInteraccion_Completado_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else int(x[-1]["tiempo"]))
    df_tiempoInteraccion_Completado_Jugador.index.name = "nivel"

    # Dataframe 2: tiempoInicio_Interaccion_Jugador
    df_tiempoInicio_Interaccion_Jugador = pd.DataFrame(tiempoInicio_Interaccion_Jugador)
    df_tiempoInicio_Interaccion_Jugador = df_tiempoInicio_Interaccion_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else int(x[0]["tiempo"]))
    df_tiempoInicio_Interaccion_Jugador.index.name = "nivel"

    # Dataframe 3: tiempoInteraccion_Interaccion_Jugador
    df_tiempoInteraccion_Interaccion_Jugador = pd.DataFrame(tiempoInteraccion_Interaccion_Jugador)
    df_tiempoInteraccion_Interaccion_Jugador = df_tiempoInteraccion_Interaccion_Jugador.applymap(lambda x: np.nan if not isinstance(x, list) or len(x) == 0 else sum(int(tiempo["tiempo"]) for tiempo in x) / len(x))
    df_tiempoInteraccion_Interaccion_Jugador.index.name = "nivel"


    return df_tiempoInteraccion_Completado_Jugador, df_tiempoInicio_Interaccion_Jugador, df_tiempoInteraccion_Interaccion_Jugador

df_tiempoInteraccion_Completado_Jugador, df_tiempoInicio_Interaccion_Jugador, df_tiempoInteraccion_Interaccion_Jugador = extraerTiemposEntreInteracciones2(rawData, True)
#df_tiempoInteraccion_Completado_Jugador
#df_tiempoInicio_Interaccion_Jugador
#df_tiempoInteraccion_Interaccion_Jugador


In [224]:
res = defaultdict(lambda : 0)

for j in resultados_Tiempos_Nivel_Jugador["erroresCod"]:
    res[j] += len(resultados_Tiempos_Nivel_Jugador["erroresCod"][j])

for j in resultados_Tiempos_Nivel_Jugador["erroresVar"]:
    res[j] += len(resultados_Tiempos_Nivel_Jugador["erroresVar"][j])

print(res)

defaultdict(<function <lambda> at 0x000001F2FA31B520>, {'pbgky': 14, 'gcpze': 15, 'srjon': 14, 'gleqe': 8, 'ftpnb': 13, 'xnpqf': 9, 'loemd': 13, 'vsxld': 9, 'zwzfv': 12, 'qeqyj': 12, 'uzzpo': 12, 'hjpbw': 13, 'ipdcp': 20, 'asopk': 18, 'pygoc': 12, 'zcibf': 9, 'rdiaf': 16, 'ubmtx': 11, 'dunul': 18, 'accul': 10, 'swvxx': 10, 'oahey': 18, 'bhyhw': 17, 'sjpkc': 13, 'ikcar': 9, 'fkmtp': 12, 'kmqou': 17, 'bdwyj': 12, 'khadt': 11, 'yfohf': 10, 'kkoif': 19, 'edxmq': 10, 'yvpmf': 11, 'stiry': 6, 'psido': 18, 'kkliq': 12, 'lwryl': 15, 'hpdxj': 7, 'tinco': 4, 'hqmhq': 8, 'ucdvy': 12, 'ewqbs': 12, 'zbgwk': 6, 'fyyya': 6, 'mymho': 4, 'rziia': 5, 'zdnso': 14, 'vvqlm': 10, 'wqmgq': 9, 'huyyb': 12, 'tavin': 12, 'yazod': 12, 'tqgyp': 19, 'jhbuv': 5, 'ahxym': 5, 'aotlm': 4, 'rqlug': 2, 'idmzi': 3, 'ptmkr': 19, 'thaho': 14, 'kzles': 13, 'yyvas': 15, 'pqwaf': 15, 'pleui': 14, 'sybwc': 7, 'ixrux': 12, 'qbsjv': 7, 'lohyo': 11, 'tcvyz': 5, 'blvkx': 7, 'qgcgc': 13, 'fmupf': 15, 'tflzv': 6, 'orjyn': 17, 'rsjxu

In [225]:
filename = 'cuestionario.csv'
data = pd.read_csv(filename, sep=';', header=0)

data.drop(['submitdate', 'G00Q01[SQ01]', 'G02Q01', 'G02Q02', 'G02Q03', 'G02Q04', 'G03Q02[other]', 'id'], axis = 'columns', inplace=True)

def mezcla_edad_genero(edad, genero):
    return genero + ' ' + str(edad)

data['edad_genero'] = data.apply(lambda x: mezcla_edad_genero(x['G03Q01'], x['G03Q02']), axis=1)

In [226]:
# Crear un DataFrame con los datos de los intentos medios de cada jugador

intentosJugadores = pd.DataFrame(intentosMedios_Individual["intentosIndividual"])
#intentosJugadores

# Convertir columnas en filas
df_stacked = intentosJugadores.stack().reset_index()
df_stacked.columns = ['nivel', 'token', 'intentos']

# Calcular promedio de intentos por jugador
promedio_intentos = df_stacked.groupby('token')['intentos'].mean()

# Crear DataFrame final
df_final = pd.DataFrame({'token': promedio_intentos.index, 'intentos': promedio_intentos.values})


# Convertir índice de df_tiempoInteraccion_Interaccion_Jugador en columna
df_tiempoInteraccion_Interaccion_Jugador2 = df_tiempoInteraccion_Interaccion_Jugador.stack().reset_index()
df_tiempoInteraccion_Interaccion_Jugador2.columns = ['nivel', 'token', 'tiempo']

# Calcular promedio de intentos por jugador
promedio_interaccion = df_tiempoInteraccion_Interaccion_Jugador2.groupby('token')['tiempo'].mean()

# Crear DataFrame final
df_final2 = pd.DataFrame({'token': promedio_interaccion.index, 'tiempo': promedio_interaccion.values})

# Unir con intentosJugadores usando merge en'token'
df_jugadores = pd.merge(df_final, df_final2, on=['token'])

# Unir con data usando merge en 'token'
df_global = pd.merge(df_jugadores, data, on='token')
#df_global
#df_jugadores

In [227]:
# Calcular los límites de los valores atípicos
# regla empírica, que establece que los valores atípicos se encuentran a más de 3 desviaciones estándar de la media.

Q1_edad = df_global['G03Q01'].quantile(0.25)
Q3_edad = df_global['G03Q01'].quantile(0.75)
IQR_edad = Q3_edad - Q1_edad
outlier_threshold_edad = Q3_edad + 1.5*IQR_edad

Q1_intentos = df_global['intentos'].quantile(0.25)
Q3_intentos = df_global['intentos'].quantile(0.75)
IQR_intentos = Q3_intentos - Q1_intentos
outlier_threshold_intentos = Q3_intentos + 1.5*IQR_intentos

Q1_tiempo = df_global['tiempo'].quantile(0.25)
Q3_tiempo = df_global['tiempo'].quantile(0.75)
IQR_tiempo = Q3_tiempo - Q1_tiempo
outlier_threshold_tiempo = Q3_tiempo + 1.5*IQR_tiempo

df = df_global[(df_global['G03Q01'] <= outlier_threshold_edad) & (df_global['intentos'] <= outlier_threshold_intentos) & (df_global['tiempo'] <= outlier_threshold_tiempo)]

# Crear una nueva columna que represente la mezcla al 50% de intentos y tiempo
df['mezcla_intentos_tiempo'] = (df['intentos'] + df['tiempo']) / 2

#df['mezcla_intentos_tiempo'] = df['intentos']*0.3 + df['tiempo']*0.7

#df['mezcla_intentos_tiempo'] = df['intentos']*0.7 + df['tiempo']*0.3


#df


C:\Users\Ángel Hortelano\AppData\Local\Temp\ipykernel_14940\2150706986.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [228]:
for i, row in df.iterrows():
    jugador = row['token']
    if jugador in res:
        df.at[i, 'errores'] = res[jugador]
    else:
        df.at[i, 'errores'] = 0

#df

C:\Users\Ángel Hortelano\AppData\Local\Temp\ipykernel_14940\3094449059.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [233]:
valUltNivel = valorUltNivelAlcanzado(ultNivelAlcanzado, tiemposMedios["listaNiveles"])

for i, row in df.iterrows():
    jugador = row['token']
    if jugador in valUltNivel:
        df.at[i, 'ultNivel'] = valUltNivel[jugador]
    else:
        df.at[i, 'ultNivel'] = 0


In [235]:
#PCA:

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

df = df.reset_index(drop=True)

# Crear un diccionario que asigne una etiqueta numérica única a cada combinación única de edad y género en la columna 'edad_genero'
etiquetas_edad_genero = {valor: indice for indice, valor in enumerate(df['edad_genero'].unique())}

# Convertir los valores de la columna 'edad_genero' en valores numéricos utilizando el diccionario de etiquetas
df['edad_genero_num'] = df['edad_genero'].apply(lambda x: etiquetas_edad_genero[x])

# Seleccionamos las columnas que se van a usar en el análisis, sin genero porque es string
columnas = ['intentos', 'tiempo', 'G03Q01', 'mezcla_intentos_tiempo', 'errores', 'edad_genero_num', 'ultNivel']

# Seleccionamos los datos que se van a usar en el análisis
data = df[columnas]

# Estandarizamos los datos
data_std = StandardScaler().fit_transform(data)

# Creamos un objeto PCA con el número de componentes que queremos obtener
pca = PCA(n_components=2)

# Aplicamos PCA a los datos estandarizados
componentes_principales = pca.fit_transform(data_std)

# Creamos un nuevo DataFrame con las componentes principales y los tokens
df_pca = pd.DataFrame(data = componentes_principales, columns = ['PC1', 'PC2'])
df_pca['token'] = df['token']

# Imprimimos el nuevo DataFrame


In [236]:
from sklearn.cluster import KMeans

#Coger las columnas necesarias para crear la matriz de características
matriz_caracteristicas = df_pca[['PC1', 'PC2']].values

# Normalizar los datos
matriz_caracteristicas_norm = StandardScaler().fit_transform(matriz_caracteristicas)

# Definimos o número de iteracións
n_iter = 20

# Obtemos os mesmos centroides iniciais que antes
SEED_VALUE = 190463  # NON CAMBIES ESTE SEED, así todos teremos os mesmos resultados
np.random.seed(SEED_VALUE)
centr_iniciais = matriz_caracteristicas_norm[np.random.choice(matriz_caracteristicas_norm.shape[0], size=5, replace=False)]

# Definimos o modelo
model = KMeans(n_clusters=len(centr_iniciais), init=centr_iniciais, n_init=1,
            max_iter=n_iter, algorithm='full', random_state=SEED_VALUE)

# Axustamos o modelo aos datos
np.random.seed(SEED_VALUE)
#df
agrupamento = model.fit(matriz_caracteristicas_norm)

# Crear la figura
fig = go.Figure(data=[go.Scatter(x=df_pca['PC1'], y=df_pca['PC2'], mode='markers',
                                 marker=dict(size=10, color=agrupamento.labels_ + 1, opacity=0.8,
                                             colorscale='Viridis'),
                                              hovertext=df['token'] + '<br>' + 'Género: ' + df['G03Q02'] + '<br>' +
                                            'Edad: ' + df['G03Q01'].astype(str) + '<br>' +
                                            'Tiempo: ' + df['tiempo'].astype(str) + '<br>' +
                                            'Intentos: ' + df['intentos'].astype(str) + '<br>' +
                                            'Ult. nivel: ' + df['ultNivel'].astype(str) + '<br>' + 
                                            'Errores: ' + df['errores'].astype(str))])

# Configurar layout
fig.update_layout(title='Clustering de intentos',
                  xaxis_title='PC1',
                  yaxis_title='PC2',
                  width=800, height=600)

# Mostrar gráfica
fig.show()


C:\Users\Ángel Hortelano\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:1366: FutureWarning:

algorithm='full' is deprecated, it will be removed in 1.3. Using 'lloyd' instead.

